In [ ]:
import torch
import numpy as np
import random
import time
from tqdm import tqdm  # 
from pathlib import Path


from models_LN_G import EPNN_LN, dataset, one_hot, cross_entropy_loss, linear, linear_d, tanh, tanh_d, sign, sign_d, sigmoid, sigmoid_d, relu, relu_d, relu6, relu6_d, softmax, softmax_d

torch.set_grad_enabled(False)


In [2]:

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
# torch.backends.cudnn.deterministic = True
# torch.backends.cudnn.benchmark = False
torch.backends.cudnn.benchmark = True


if torch.cuda.is_available():
    print(f"cuda:True")
    for i in range(torch.cuda.device_count()):
        device = torch.device(f"cuda:{i}")
        properties = torch.cuda.get_device_properties(device)
        print(f"GPU {i}:{properties.name}:{properties.total_memory/1024/1024/1024:.2f}GB")
else:
    print(f"cuda:False")
    device = torch.device("cpu")

cuda:True
GPU 0:NVIDIA GeForce RTX 4070:11.99GB


In [3]:

#========================================================
# W_conn = [wij]
# i row: input 
# j col: output
# 0:input layer     ...:hidden layer    -1:output layer
#--------------------------------------------------------
def make_W_conn(n_hidden, startres=-1, block_size=2):
    """
    Generate the connectivity matrix W_conn:
      - Total nodes N = 1 (input) + n_hidden + 1 (output)
      - Bidirectional connections between adjacent layers (i <-> i+1)
      - Residual connections: from the start of each block, skip one block and connect forward
        (for block_size=2, startres=1, that is i -> i+3, e.g. 2->5, 4->7, 6->9…)
    """
    N = n_hidden + 2
    W = np.zeros((N, N), dtype=int)

    # connect each layer to the next (both directions)
    for i in range(N - 1):
        W[i, i + 1] = 1
        W[i + 1, i] = 1

    W[1, 0]=0
    # add residual (skip) connections
    if startres>0:
        for start in range(startres, N, block_size):
            target = start + block_size 
            if target < N:
                W[start, target] = 1
                W[target, start] = 1

    return W

In [4]:

NL = 10
W_conn = make_W_conn(n_hidden=NL, startres=1, block_size=3)

nodes = [28*28] + [64]*NL + [10]
print(W_conn)

print(f"W_conn_symm:{np.array_equal(W_conn[1:,1:], W_conn[1:,1:].T)}")
W_conn[1:,1:] = np.triu(W_conn[1:,1:]) + np.triu(W_conn[1:,1:], 1).T
# -------------------
Para_EP = {
     'EP_symm_W': True,
     'fsc': 1,
     'bsc': 0.1,
     'nudge': 0.1,
     'It2sta': (len(nodes)-2)*5,
     'It2sta2': (len(nodes)-2)*2,
     }

# training:
Para_t = {
     'f': tanh,
     'fd': tanh_d,
     'ff': softmax,
     'ffd': softmax_d,

     'adam': True,
     'eta': 1e-3,
     'epoch': 50,
     'batchsize': 500,
     'task': "MNIST",
     'method': 'EP' ,
     
     'bias_learn': False,
     'bias_init_sc':0,
     }

[[0 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 1 0 0 0 0 0 0 0]
 [0 1 0 1 0 0 0 0 0 0 0 0]
 [0 0 1 0 1 0 0 0 0 0 0 0]
 [0 1 0 1 0 1 0 1 0 0 0 0]
 [0 0 0 0 1 0 1 0 0 0 0 0]
 [0 0 0 0 0 1 0 1 0 0 0 0]
 [0 0 0 0 1 0 1 0 1 0 1 0]
 [0 0 0 0 0 0 0 1 0 1 0 0]
 [0 0 0 0 0 0 0 0 1 0 1 0]
 [0 0 0 0 0 0 0 1 0 1 0 1]
 [0 0 0 0 0 0 0 0 0 0 1 0]]
W_conn_symm:True


In [5]:
# 
fsave = True

##
Task_info = f"First_symm_10HL_Res-Task2-0"
# Task_info = f"Test"
Einfo = ''

res_path = Path(f"./Res/{Task_info}")
M_range = ["FMNIST","MNIST"] #"CIFAR10","MNIST"
ParaRange = [0.01,0.1] # 0.01,0.1
multitest = 5

In [6]:
res_path.mkdir(parents=True, exist_ok=True)
res = np.zeros((len(M_range),len(ParaRange),(multitest),6,Para_t['epoch']))

zall_list = [[[] for _ in range(len(ParaRange))] for _ in range(len(M_range))]
eall_list = [[[[] for _ in range((multitest))] for _ in range(len(ParaRange))] for _ in range(len(M_range))]


In [7]:
for iM in range( len(M_range) ):
    for iPara in range( len(ParaRange) ):
        for  imultitest in range((multitest)):
            Para_EP['bsc'] = ParaRange[iPara]
            Para_t['task'] = M_range[iM]

            (_, [train_loader_gpu,valid_loader_gpu,test_loader_gpu], [nodes[0], nodes[-1]]
                 ) = dataset(Para_t['task'], device=device, bs=Para_t['batchsize'], validbstimes=4 )
            #-----------------
            Para_EP['It2sta'] = (len(nodes)-2)*10
            Para_EP['It2sta2'] = (len(nodes)-2)*5
            # ---------------------------------
            model = EPNN_LN(W_conn, nodes=nodes, Para_t=Para_t, Para_EP=Para_EP, device=device).to(device)

            s_task_info = f"{Task_info}-{Para_t['method']}_L{(len(nodes)-2)}--{Para_t['task']}_EP_b_sc{Para_EP['bsc']}-{Einfo}"
            print(s_task_info)

            ## Train

            for epoch in range(Para_t['epoch']):
                losssum = 0
                random.shuffle(train_loader_gpu)
                with tqdm(total=len(train_loader_gpu), desc=f"Epoch {epoch + 1}/{Para_t['epoch']}:", unit='batch', ncols=90, mininterval=1, colour='blue') as pbar:
                    for i, (images, labels) in enumerate(train_loader_gpu):
                        # 
                        images = images.view(-1, nodes[0])
                        labels_one_hot = one_hot(labels, nodes[-1], device=device)
                        # 
                        outputs = model.forward(images)
                        # 
                        loss = cross_entropy_loss(outputs, labels_one_hot.to(device))
                        losssum += loss.item()
                        #
                        outputs = model.backward(images, labels_one_hot, outputs)
                        # 
                        model.update_weights_adam() if Para_t['adam'] else model.update_weights() #model.update_weights_adam() model.update_weights()
                        # 
                        if (i+1) % 10 == 0: 
                            pbar.set_postfix({'loss': f'{losssum/(i+1):.6f}'})  
                            pbar.update(10)

                # SR, ftmle, mle, zall = model.Lyapunov_EP(images[-1:,:], y=None, t_e=500, ret_zall=True)
                # if imultitest==0: zall_list[iM][iPara].append(zall)

                # terr = model.ret_error() if imultitest==0 else 0
                # eall_list[iM][iPara][imultitest].append(terr)

                # train set ------------------------------------------------------------
                train_correct = 0
                train_total = 0
                losssum = 0
                for images, labels in valid_loader_gpu:
                    images = images.view(-1, nodes[0])
                    labels_one_hot = one_hot(labels, nodes[-1], device=device)
                    # 
                    outputs = model.forward(images)

                    _, predicted = torch.max(outputs, 1)
                    loss = cross_entropy_loss(outputs, labels_one_hot.to(device))
                    losssum += loss.item()
                    train_total += labels.size(0)
                    train_correct += (predicted == labels).sum().item()

                train_accuracy = train_correct / train_total

                # test set -------------------------------------------------------------
                test_correct = 0
                test_total = 0
                # ---------
                for images, labels in test_loader_gpu:
                    images = images.view(-1, nodes[0])
                    # 
                    outputs = model.forward(images)

                    _, predicted = torch.max(outputs, 1)
                    test_total += labels.size(0)
                    test_correct += (predicted == labels).sum().item()

                test_accuracy = test_correct / test_total

                res[iM][iPara][imultitest][0][epoch] = losssum
                res[iM][iPara][imultitest][1][epoch] = train_accuracy
                res[iM][iPara][imultitest][2][epoch] = test_accuracy
                # res[iM][iPara][imultitest][3][epoch] = ftmle
                # res[iM][iPara][imultitest][4][epoch] = mle
                # res[iM][iPara][imultitest][5][epoch] = SR

                current_time = time.strftime("%Y%m%d-%H%M%S", time.localtime())
                print(f'\t Loss: {res[iM][iPara][imultitest][0][epoch] :.3f},\t'
                    f'Train : {res[iM][iPara][imultitest][1][epoch] * 100:.2f}%({np.max(res[iM][iPara][imultitest][1]) * 100:.2f}%),\t'
                    f'Test : {res[iM][iPara][imultitest][2][epoch] * 100:.2f}%({np.max(res[iM][iPara][imultitest][2]) * 100:.2f}%),\t'
                    f'Current Time: {current_time}')
                
                # if 1-train_accuracy <1e-4:
                #     break
            
            # best result
            print(f"{s_task_info} Time: {current_time}, Epochs: {Para_t['epoch']}, Learning Rate: {Para_t['eta']}, "
                f"Best: {max(res[iM][iPara][imultitest][1]) * 100:.2f}%  {max(res[iM][iPara][imultitest][2]) * 100:.2f}%\n")
            
        for attempt in range(5):
            try:
                with open(res_path/"res.txt", "a") as f:
                    trainacct = np.max(res[iM][iPara][:,1,:], axis=-1)
                    testacct = np.max(res[iM][iPara][:,2,:], axis=-1)
                    f.write(f"{s_task_info} Time: {current_time}: "
                            f"train: {np.mean(trainacct) * 100:.2f}%+-{np.std(trainacct, ddof=1)*100:.2f}% ({np.min(trainacct) * 100:.2f}% {np.max(trainacct) * 100:.2f}%)"
                            f"\t test: {np.mean(testacct) * 100:.2f}%+-{np.std(testacct, ddof=1)*100:.2f}% ({np.min(testacct) * 100:.2f}% {np.max(testacct) * 100:.2f}%)\n")

                    if Para_EP!=None:
                        for key, value in Para_EP.items():
                            f.write(f"{key}: {value}  ")
                    f.write(f'\n\n')  

                break  # 
            except PermissionError:
                if attempt < 5 - 1:
                    print(f"Permission denied, retrying in {5} seconds...")
                    time.sleep(5)
                else:
                    raise  # 

First_symm_10HL_Res-Task2-0-EP_L10--FMNIST_EP_b_sc0.01-


Epoch 1/50:: 100%|████████████████████| 120/120 [00:02<00:00, 46.91batch/s, loss=0.836103]


	 Loss: 15.482,	Train : 82.34%(82.34%),	Test : 81.25%(81.25%),	Current Time: 20250720-125134


Epoch 2/50:: 100%|████████████████████| 120/120 [00:02<00:00, 48.97batch/s, loss=0.491268]


	 Loss: 14.524,	Train : 83.56%(83.56%),	Test : 82.31%(82.31%),	Current Time: 20250720-125137


...

	 Loss: 0.250,	Train : 99.74%(99.84%),	Test : 97.23%(97.38%),	Current Time: 20250720-134209


Epoch 49/50:: 100%|███████████████████| 120/120 [00:02<00:00, 49.68batch/s, loss=0.011270]


	 Loss: 0.112,	Train : 99.88%(99.88%),	Test : 97.45%(97.45%),	Current Time: 20250720-134212


Epoch 50/50:: 100%|███████████████████| 120/120 [00:02<00:00, 49.54batch/s, loss=0.006467]


	 Loss: 0.093,	Train : 99.91%(99.91%),	Test : 97.46%(97.46%),	Current Time: 20250720-134215
First_symm_10HL_Res-Task2-0-EP_L10--MNIST_EP_b_sc0.1- Time: 20250720-134215, Epochs: 50, Learning Rate: 0.001, Best: 99.91%  97.46%



In [8]:
current_time = time.strftime("%Y%m%d-%H%M%S", time.localtime())
if fsave: 
    save_dict = {
        "M_range": M_range,
        "ParaRange": ParaRange,
        "multitest": multitest,
        "Para_EP": Para_EP,
        "Para_t": Para_t,       
        "res": res,
    }
    np.savez_compressed(res_path / f"{s_task_info}_{current_time}_res",  **save_dict)

    # save_dict = {
    #     f"eall_{i}_{j}_{k}": np.array(eall_list[i][j][k])  # 
    #     for i in range(len(M_range))
    #     for j in range(len(ParaRange))
    #     for k in range(multitest)
    # }
    # np.savez_compressed(res_path / f"{s_task_info}_{current_time}_E", **save_dict)

    # save_dict = {
    #     f"zall_{i}_{j}": np.array(zall_list[i][j])  # 
    #     for i in range(len(M_range))
    #     for j in range(len(ParaRange))
    # }
    # np.savez_compressed(res_path / f"{s_task_info}_{current_time}_dyn", **save_dict)


In [9]:

for iM in range( len(M_range) ):
    for iPara in range( len(ParaRange) ):        
        tt = res[iM][iPara]
        testacctt = np.max(tt[:,2,:], axis= -1)

        tt = res[iM][iPara]
        trainacctt = np.max(tt[:,1,:], axis= -1)

        print(f"{Para_t['task']}\t_{M_range[iM]}\t_{ParaRange[iPara]}\t_"
              f"train: {np.mean(trainacctt) * 100:.2f}%+-{np.std(trainacctt, ddof=1)*100:.2f}% ({np.min(trainacctt) * 100:.2f}% {np.max(trainacctt) * 100:.2f}%)"
              f"\t test: {np.mean(testacctt) * 100:.2f}%+-{np.std(testacctt, ddof=1)*100:.2f}% ({np.min(testacctt) * 100:.2f}% {np.max(testacctt) * 100:.2f}%)\n")
        

print('0')

MNIST	_FMNIST	_0.01	_train: 92.30%+-0.28% (91.90% 92.59%)	 test: 88.33%+-0.22% (88.06% 88.67%)

MNIST	_FMNIST	_0.1	_train: 93.48%+-0.69% (92.55% 94.21%)	 test: 88.47%+-0.22% (88.10% 88.61%)

MNIST	_MNIST	_0.01	_train: 99.76%+-0.05% (99.70% 99.82%)	 test: 97.52%+-0.06% (97.45% 97.60%)

MNIST	_MNIST	_0.1	_train: 99.88%+-0.04% (99.83% 99.92%)	 test: 97.52%+-0.09% (97.44% 97.65%)

0
